In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#session = tf.Session(config=config)
set_session(tf.Session(config=config))

['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [4]:
import threading

In [5]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [6]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [7]:
import datetime
import os

In [8]:
import pandas as pd
import h5py

In [9]:
# Setup data seed
np.random.seed(3254)

In [10]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [11]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [12]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [13]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/batchtest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 400
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
#    for label in [3904, 3910, 3912, 3913, 3914, 3929]:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3904
{3904: 1.0}
composition with label != 0
Y_train 0.17552334943639292 0.003517512077294686
composition with label != 0
Y_val 0.21428571428571427 0.005008928571428571
composition with label != 0
Y_test 0.1791907514450867 0.003717485549132948
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., epochs=50, callbacks=[<keras.ca..., verbose=1, steps_per_epoch=400)`


Epoch 1/50
400/400 [==============================] - 46s 115ms/step - loss: 4.7602 - binary_crossentropy: 0.0580 - jaccard_coef_int: 0.0328 - binary_accuracy: 0.9898 - val_loss: 5.5452 - val_binary_crossentropy: 0.1347 - val_jaccard_coef_int: 0.0051 - val_binary_accuracy: 0.9834
Epoch 2/50
400/400 [==============================] - 39s 98ms/step - loss: 3.3754 - binary_crossentropy: 0.0544 - jaccard_coef_int: 0.0559 - binary_accuracy: 0.9912 - val_loss: 11.4504 - val_binary_crossentropy: 4.1892 - val_jaccard_coef_int: 0.0054 - val_binary_accuracy: 0.6122
Epoch 3/50
400/400 [==============================] - 39s 98ms/step - loss: 3.6191 - binary_crossentropy: 0.0837 - jaccard_coef_int: 0.0431 - binary_accuracy: 0.9884 - val_loss: 2.8539 - val_binary_crossentropy: 0.1493 - val_jaccard_coef_int: 0.0751 - val_binary_accuracy: 0.9849
Epoch 4/50
400/400 [==============================] - 39s 99ms/step - loss: 3.3392 - binary_crossentropy: 0.0718 - jaccard_coef_int: 0.0557 - binary_accuracy:

Epoch 7/50
400/400 [==============================] - 50s 125ms/step - loss: 2.1062 - binary_crossentropy: 0.2729 - jaccard_coef_int: 0.1966 - binary_accuracy: 0.9716 - val_loss: 4.2088 - val_binary_crossentropy: 0.1298 - val_jaccard_coef_int: 0.0181 - val_binary_accuracy: 0.9868
Epoch 8/50
400/400 [==============================] - 50s 124ms/step - loss: 2.1005 - binary_crossentropy: 0.2792 - jaccard_coef_int: 0.1992 - binary_accuracy: 0.9708 - val_loss: 2.4051 - val_binary_crossentropy: 0.3105 - val_jaccard_coef_int: 0.1496 - val_binary_accuracy: 0.9688
Epoch 9/50
400/400 [==============================] - 50s 125ms/step - loss: 2.0254 - binary_crossentropy: 0.2777 - jaccard_coef_int: 0.2101 - binary_accuracy: 0.9721 - val_loss: 3.0602 - val_binary_crossentropy: 0.1280 - val_jaccard_coef_int: 0.0573 - val_binary_accuracy: 0.9870
Epoch 10/50
400/400 [==============================] - 50s 125ms/step - loss: 2.0003 - binary_crossentropy: 0.2787 - jaccard_coef_int: 0.2135 - binary_accura

Epoch 13/50
400/400 [==============================] - 50s 125ms/step - loss: 1.8143 - binary_crossentropy: 0.2513 - jaccard_coef_int: 0.2318 - binary_accuracy: 0.9790 - val_loss: 3.2076 - val_binary_crossentropy: 0.3112 - val_jaccard_coef_int: 0.0605 - val_binary_accuracy: 0.9745
Epoch 14/50
400/400 [==============================] - 50s 125ms/step - loss: 1.6896 - binary_crossentropy: 0.2434 - jaccard_coef_int: 0.2575 - binary_accuracy: 0.9801 - val_loss: 3.5264 - val_binary_crossentropy: 0.2644 - val_jaccard_coef_int: 0.0411 - val_binary_accuracy: 0.9787
Epoch 15/50
400/400 [==============================] - 50s 125ms/step - loss: 1.5951 - binary_crossentropy: 0.2350 - jaccard_coef_int: 0.2778 - binary_accuracy: 0.9811 - val_loss: 3.2401 - val_binary_crossentropy: 0.2883 - val_jaccard_coef_int: 0.0559 - val_binary_accuracy: 0.9776
Epoch 16/50
400/400 [==============================] - 50s 125ms/step - loss: 1.5244 - binary_crossentropy: 0.2268 - jaccard_coef_int: 0.2933 - binary_acc

Epoch 19/50
400/400 [==============================] - 39s 99ms/step - loss: 1.4953 - binary_crossentropy: 0.4974 - jaccard_coef_int: 0.4291 - binary_accuracy: 0.9597 - val_loss: 5.3334 - val_binary_crossentropy: 0.8048 - val_jaccard_coef_int: 0.0134 - val_binary_accuracy: 0.9363
Epoch 20/50
400/400 [==============================] - 39s 99ms/step - loss: 1.4593 - binary_crossentropy: 0.4928 - jaccard_coef_int: 0.4375 - binary_accuracy: 0.9607 - val_loss: 4.9561 - val_binary_crossentropy: 0.8863 - val_jaccard_coef_int: 0.0205 - val_binary_accuracy: 0.9332
Epoch 21/50
400/400 [==============================] - 40s 99ms/step - loss: 1.4298 - binary_crossentropy: 0.4921 - jaccard_coef_int: 0.4464 - binary_accuracy: 0.9613 - val_loss: 5.5014 - val_binary_crossentropy: 0.8651 - val_jaccard_coef_int: 0.0116 - val_binary_accuracy: 0.9349
Epoch 22/50
400/400 [==============================] - 40s 99ms/step - loss: 1.3888 - binary_crossentropy: 0.4903 - jaccard_coef_int: 0.4584 - binary_accurac

Epoch 25/50
400/400 [==============================] - 40s 100ms/step - loss: 1.8375 - binary_crossentropy: 0.6620 - jaccard_coef_int: 0.3259 - binary_accuracy: 0.9557 - val_loss: 4.7728 - val_binary_crossentropy: 1.2672 - val_jaccard_coef_int: 0.0324 - val_binary_accuracy: 0.9157
Epoch 26/50
400/400 [==============================] - 40s 99ms/step - loss: 1.7927 - binary_crossentropy: 0.6571 - jaccard_coef_int: 0.3374 - binary_accuracy: 0.9564 - val_loss: 5.1556 - val_binary_crossentropy: 1.2193 - val_jaccard_coef_int: 0.0206 - val_binary_accuracy: 0.9207
Epoch 27/50
400/400 [==============================] - 40s 100ms/step - loss: 1.7984 - binary_crossentropy: 0.6629 - jaccard_coef_int: 0.3361 - binary_accuracy: 0.9562 - val_loss: 5.5683 - val_binary_crossentropy: 1.1676 - val_jaccard_coef_int: 0.0128 - val_binary_accuracy: 0.9247
Epoch 28/50
400/400 [==============================] - 40s 100ms/step - loss: 1.7584 - binary_crossentropy: 0.6518 - jaccard_coef_int: 0.3444 - binary_accu

Epoch 2/50
400/400 [==============================] - 40s 100ms/step - loss: 2.8860 - binary_crossentropy: 0.0777 - jaccard_coef_int: 0.1448 - binary_accuracy: 0.9842 - val_loss: 10.5147 - val_binary_crossentropy: 0.0083 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 3/50
400/400 [==============================] - 40s 100ms/step - loss: 2.8306 - binary_crossentropy: 0.0907 - jaccard_coef_int: 0.1384 - binary_accuracy: 0.9835 - val_loss: 6.2960 - val_binary_crossentropy: 0.0796 - val_jaccard_coef_int: 0.0031 - val_binary_accuracy: 0.9764
Epoch 4/50
400/400 [==============================] - 40s 100ms/step - loss: 2.7992 - binary_crossentropy: 0.0826 - jaccard_coef_int: 0.1402 - binary_accuracy: 0.9865 - val_loss: 10.7589 - val_binary_crossentropy: 0.0062 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 5/50
400/400 [==============================] - 40s 100ms/step - loss: 2.6570 - binary_crossentropy: 0.0905 - jaccard_coef_int: 0.1587 - binary_accur

In [28]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchtest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 400
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
#    for label in list_final:
    for label in [98,99]:

        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 98
{98: 1.0}
composition with label != 0
Y_train 0.05958132045088567 0.014111312399355877
composition with label != 0
Y_val 0.05714285714285714 0.010928571428571428
composition with label != 0
Y_test 0.046242774566473986 0.010343208092485548
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=50, steps_per_epoch=400, validation_data=(array([[[..., verbose=1, callbacks=[<keras.ca...)`


Epoch 1/50
400/400 [==============================] - 56s 139ms/step - loss: 1.4551 - binary_crossentropy: 0.0490 - jaccard_coef_int: 0.6190 - binary_accuracy: 0.9916 - val_loss: 1.1514 - val_binary_crossentropy: 0.0918 - val_jaccard_coef_int: 0.4301 - val_binary_accuracy: 0.9861
Epoch 2/50
400/400 [==============================] - 40s 100ms/step - loss: 0.7505 - binary_crossentropy: 0.0459 - jaccard_coef_int: 0.6342 - binary_accuracy: 0.9930 - val_loss: 1.1023 - val_binary_crossentropy: 0.0864 - val_jaccard_coef_int: 0.4284 - val_binary_accuracy: 0.9862
Epoch 3/50
400/400 [==============================] - 40s 100ms/step - loss: 0.6449 - binary_crossentropy: 0.0504 - jaccard_coef_int: 0.6503 - binary_accuracy: 0.9933 - val_loss: 1.0737 - val_binary_crossentropy: 0.1173 - val_jaccard_coef_int: 0.4311 - val_binary_accuracy: 0.9862
Epoch 4/50
400/400 [==============================] - 40s 100ms/step - loss: 0.6262 - binary_crossentropy: 0.0513 - jaccard_coef_int: 0.6517 - binary_accurac

Epoch 7/50
400/400 [==============================] - 41s 101ms/step - loss: 3.5963 - binary_crossentropy: 0.1815 - jaccard_coef_int: 0.0547 - binary_accuracy: 0.9805 - val_loss: 3.9278 - val_binary_crossentropy: 0.1470 - val_jaccard_coef_int: 0.0284 - val_binary_accuracy: 0.9769
Epoch 8/50
400/400 [==============================] - 41s 102ms/step - loss: 3.4333 - binary_crossentropy: 0.1180 - jaccard_coef_int: 0.0614 - binary_accuracy: 0.9859 - val_loss: 4.4755 - val_binary_crossentropy: 0.1371 - val_jaccard_coef_int: 0.0151 - val_binary_accuracy: 0.9834
Epoch 9/50
400/400 [==============================] - 41s 102ms/step - loss: 3.7271 - binary_crossentropy: 0.2705 - jaccard_coef_int: 0.0537 - binary_accuracy: 0.9759 - val_loss: 3.2057 - val_binary_crossentropy: 0.4434 - val_jaccard_coef_int: 0.0737 - val_binary_accuracy: 0.9641
Epoch 10/50
400/400 [==============================] - 41s 102ms/step - loss: 3.6672 - binary_crossentropy: 0.2314 - jaccard_coef_int: 0.0555 - binary_accura

In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchtest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 400
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=50, verbose=1, steps_per_epoch=400)`


Epoch 1/50
400/400 [==============================] - 47s 118ms/step - loss: 1.2145 - binary_crossentropy: 0.0373 - jaccard_coef_int: 0.7217 - binary_accuracy: 0.9934 - val_loss: 0.6508 - val_binary_crossentropy: 0.0602 - val_jaccard_coef_int: 0.6565 - val_binary_accuracy: 0.9910
Epoch 2/50
400/400 [==============================] - 39s 98ms/step - loss: 0.4414 - binary_crossentropy: 0.0287 - jaccard_coef_int: 0.7543 - binary_accuracy: 0.9953 - val_loss: 0.5837 - val_binary_crossentropy: 0.0769 - val_jaccard_coef_int: 0.6600 - val_binary_accuracy: 0.9910
Epoch 3/50
400/400 [==============================] - 40s 100ms/step - loss: 0.4036 - binary_crossentropy: 0.0327 - jaccard_coef_int: 0.7541 - binary_accuracy: 0.9953 - val_loss: 0.5629 - val_binary_crossentropy: 0.0785 - val_jaccard_coef_int: 0.6632 - val_binary_accuracy: 0.9911
Epoch 4/50
400/400 [==============================] - 39s 98ms/step - loss: 0.3734 - binary_crossentropy: 0.0351 - jaccard_coef_int: 0.7602 - binary_accuracy:

Epoch 7/50
400/400 [==============================] - 40s 100ms/step - loss: 4.2985 - binary_crossentropy: 1.6319 - jaccard_coef_int: 0.3225 - binary_accuracy: 0.8025 - val_loss: 4.6585 - val_binary_crossentropy: 1.7113 - val_jaccard_coef_int: 0.2556 - val_binary_accuracy: 0.7942
Epoch 8/50
400/400 [==============================] - 40s 100ms/step - loss: 4.1667 - binary_crossentropy: 1.6682 - jaccard_coef_int: 0.3378 - binary_accuracy: 0.8078 - val_loss: 4.9180 - val_binary_crossentropy: 1.9266 - val_jaccard_coef_int: 0.2102 - val_binary_accuracy: 0.7902
Epoch 9/50
400/400 [==============================] - 40s 100ms/step - loss: 4.0358 - binary_crossentropy: 1.6997 - jaccard_coef_int: 0.3490 - binary_accuracy: 0.8139 - val_loss: 4.9207 - val_binary_crossentropy: 1.9327 - val_jaccard_coef_int: 0.1923 - val_binary_accuracy: 0.7953
Epoch 10/50
400/400 [==============================] - 40s 100ms/step - loss: 3.8679 - binary_crossentropy: 1.7109 - jaccard_coef_int: 0.3647 - binary_accura

Epoch 13/50
400/400 [==============================] - 40s 101ms/step - loss: 2.1066 - binary_crossentropy: 1.0374 - jaccard_coef_int: 0.4994 - binary_accuracy: 0.9119 - val_loss: 4.0567 - val_binary_crossentropy: 2.0357 - val_jaccard_coef_int: 0.2418 - val_binary_accuracy: 0.8354
Epoch 14/50
400/400 [==============================] - 40s 100ms/step - loss: 2.0444 - binary_crossentropy: 1.0323 - jaccard_coef_int: 0.5119 - binary_accuracy: 0.9142 - val_loss: 4.0247 - val_binary_crossentropy: 2.0703 - val_jaccard_coef_int: 0.2497 - val_binary_accuracy: 0.8354
Epoch 15/50
400/400 [==============================] - 40s 100ms/step - loss: 1.9710 - binary_crossentropy: 1.0131 - jaccard_coef_int: 0.5249 - binary_accuracy: 0.9172 - val_loss: 4.1936 - val_binary_crossentropy: 2.2199 - val_jaccard_coef_int: 0.2265 - val_binary_accuracy: 0.8309
Epoch 16/50
400/400 [==============================] - 41s 101ms/step - loss: 1.9022 - binary_crossentropy: 0.9823 - jaccard_coef_int: 0.5353 - binary_acc

...

400/400 [==============================] - 41s 102ms/step - loss: 3.2442 - binary_crossentropy: 0.0974 - jaccard_coef_int: 0.0635 - binary_accuracy: 0.9875 - val_loss: 5.5710 - val_binary_crossentropy: 0.0783 - val_jaccard_coef_int: 0.0041 - val_binary_accuracy: 0.9913
Epoch 8/50
400/400 [==============================] - 40s 101ms/step - loss: 3.0832 - binary_crossentropy: 0.1073 - jaccard_coef_int: 0.0717 - binary_accuracy: 0.9884 - val_loss: 4.7573 - val_binary_crossentropy: 0.0507 - val_jaccard_coef_int: 0.0598 - val_binary_accuracy: 0.9946
Epoch 9/50
400/400 [==============================] - 40s 100ms/step - loss: 2.7686 - binary_crossentropy: 0.0894 - jaccard_coef_int: 0.0967 - binary_accuracy: 0.9912 - val_loss: 2.7069 - val_binary_crossentropy: 0.0603 - val_jaccard_coef_int: 0.0760 - val_binary_accuracy: 0.9935
Epoch 10/50
400/400 [==============================] - 40s 100ms/step - loss: 3.2036 - binary_crossentropy: 0.1024 - jaccard_coef_int: 0.0698 - binary_accuracy: 0.9866 

Epoch 13/50
400/400 [==============================] - 49s 123ms/step - loss: 1.0092 - binary_crossentropy: 0.5315 - jaccard_coef_int: 0.8636 - binary_accuracy: 0.9463 - val_loss: 2.5744 - val_binary_crossentropy: 1.3750 - val_jaccard_coef_int: 0.6812 - val_binary_accuracy: 0.8635
Epoch 14/50
400/400 [==============================] - 49s 123ms/step - loss: 0.9734 - binary_crossentropy: 0.5306 - jaccard_coef_int: 0.8684 - binary_accuracy: 0.9482 - val_loss: 2.5223 - val_binary_crossentropy: 1.4261 - val_jaccard_coef_int: 0.6900 - val_binary_accuracy: 0.8659
Epoch 15/50
400/400 [==============================] - 49s 123ms/step - loss: 0.9470 - binary_crossentropy: 0.5319 - jaccard_coef_int: 0.8706 - binary_accuracy: 0.9497 - val_loss: 2.5265 - val_binary_crossentropy: 1.4737 - val_jaccard_coef_int: 0.6879 - val_binary_accuracy: 0.8658
Epoch 16/50
400/400 [==============================] - 49s 123ms/step - loss: 0.9254 - binary_crossentropy: 0.5375 - jaccard_coef_int: 0.8733 - binary_acc

Epoch 19/50
400/400 [==============================] - 40s 100ms/step - loss: 1.3196 - binary_crossentropy: 0.5088 - jaccard_coef_int: 0.5103 - binary_accuracy: 0.9598 - val_loss: 3.3494 - val_binary_crossentropy: 1.2589 - val_jaccard_coef_int: 0.1632 - val_binary_accuracy: 0.9041
Epoch 20/50
400/400 [==============================] - 40s 100ms/step - loss: 1.2818 - binary_crossentropy: 0.5027 - jaccard_coef_int: 0.5189 - binary_accuracy: 0.9611 - val_loss: 3.2506 - val_binary_crossentropy: 1.1420 - val_jaccard_coef_int: 0.1581 - val_binary_accuracy: 0.9121
Epoch 21/50
400/400 [==============================] - 40s 101ms/step - loss: 1.2427 - binary_crossentropy: 0.5004 - jaccard_coef_int: 0.5309 - binary_accuracy: 0.9621 - val_loss: 3.4593 - val_binary_crossentropy: 1.2692 - val_jaccard_coef_int: 0.1350 - val_binary_accuracy: 0.9088
Epoch 22/50
400/400 [==============================] - 40s 100ms/step - loss: 1.2219 - binary_crossentropy: 0.5060 - jaccard_coef_int: 0.5378 - binary_acc

400/400 [==============================] - 40s 100ms/step - loss: 2.0632 - binary_crossentropy: 0.0998 - jaccard_coef_int: 0.2922 - binary_accuracy: 0.9884 - val_loss: 23.7898 - val_binary_crossentropy: 0.0160 - val_jaccard_coef_int: 0.6375 - val_binary_accuracy: 0.9943
Epoch 25/50
400/400 [==============================] - 40s 100ms/step - loss: 1.9099 - binary_crossentropy: 0.0747 - jaccard_coef_int: 0.2903 - binary_accuracy: 0.9896 - val_loss: 24.7333 - val_binary_crossentropy: 4.4665e-04 - val_jaccard_coef_int: 0.9375 - val_binary_accuracy: 1.0000
Epoch 26/50
400/400 [==============================] - 40s 100ms/step - loss: 1.6201 - binary_crossentropy: 0.0740 - jaccard_coef_int: 0.3528 - binary_accuracy: 0.9915 - val_loss: 32.9255 - val_binary_crossentropy: 0.2313 - val_jaccard_coef_int: 7.6845e-15 - val_binary_accuracy: 0.9743
Epoch 27/50
400/400 [==============================] - 40s 99ms/step - loss: 1.4023 - binary_crossentropy: 0.0767 - jaccard_coef_int: 0.3770 - binary_accur

Epoch 1/50
400/400 [==============================] - 57s 142ms/step - loss: 3.6543 - binary_crossentropy: 0.0587 - jaccard_coef_int: 0.1131 - binary_accuracy: 0.9890 - val_loss: 4.2040 - val_binary_crossentropy: 0.0729 - val_jaccard_coef_int: 0.0195 - val_binary_accuracy: 0.9869
Epoch 2/50
400/400 [==============================] - 40s 100ms/step - loss: 2.0177 - binary_crossentropy: 0.0436 - jaccard_coef_int: 0.1784 - binary_accuracy: 0.9933 - val_loss: 2.8223 - val_binary_crossentropy: 0.0180 - val_jaccard_coef_int: 0.0646 - val_binary_accuracy: 0.9975
Epoch 3/50
400/400 [==============================] - 40s 99ms/step - loss: 1.9553 - binary_crossentropy: 0.0508 - jaccard_coef_int: 0.1870 - binary_accuracy: 0.9937 - val_loss: 4.7629 - val_binary_crossentropy: 0.5576 - val_jaccard_coef_int: 0.0190 - val_binary_accuracy: 0.9506
Epoch 4/50
400/400 [==============================] - 40s 99ms/step - loss: 1.8816 - binary_crossentropy: 0.0512 - jaccard_coef_int: 0.1952 - binary_accuracy:

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchtest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 400
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=50, validation_data=(array([[[..., verbose=1, steps_per_epoch=400, callbacks=[<keras.ca...)`


Epoch 1/50
400/400 [==============================] - 46s 116ms/step - loss: 2.5688 - binary_crossentropy: 0.2596 - jaccard_coef_int: 0.4212 - binary_accuracy: 0.9228 - val_loss: 3.4696 - val_binary_crossentropy: 0.4699 - val_jaccard_coef_int: 0.2014 - val_binary_accuracy: 0.8963
Epoch 2/50
400/400 [==============================] - 39s 98ms/step - loss: 1.8465 - binary_crossentropy: 0.3283 - jaccard_coef_int: 0.5016 - binary_accuracy: 0.9316 - val_loss: 3.7508 - val_binary_crossentropy: 1.2428 - val_jaccard_coef_int: 0.3391 - val_binary_accuracy: 0.8352
Epoch 3/50
400/400 [==============================] - 39s 98ms/step - loss: 1.4524 - binary_crossentropy: 0.2699 - jaccard_coef_int: 0.5764 - binary_accuracy: 0.9466 - val_loss: 0.9909 - val_binary_crossentropy: 0.1527 - val_jaccard_coef_int: 0.6709 - val_binary_accuracy: 0.9655
Epoch 4/50
400/400 [==============================] - 39s 98ms/step - loss: 1.0124 - binary_crossentropy: 0.1704 - jaccard_coef_int: 0.6772 - binary_accuracy: 

Epoch 7/50
400/400 [==============================] - 40s 100ms/step - loss: 0.6172 - binary_crossentropy: 0.0808 - jaccard_coef_int: 0.7218 - binary_accuracy: 0.9832 - val_loss: 0.5587 - val_binary_crossentropy: 0.0706 - val_jaccard_coef_int: 0.7518 - val_binary_accuracy: 0.9840
Epoch 8/50
400/400 [==============================] - 40s 100ms/step - loss: 0.5732 - binary_crossentropy: 0.0790 - jaccard_coef_int: 0.7393 - binary_accuracy: 0.9842 - val_loss: 0.6192 - val_binary_crossentropy: 0.0898 - val_jaccard_coef_int: 0.7397 - val_binary_accuracy: 0.9813
Epoch 9/50
400/400 [==============================] - 40s 100ms/step - loss: 0.5128 - binary_crossentropy: 0.0713 - jaccard_coef_int: 0.7626 - binary_accuracy: 0.9859 - val_loss: 0.4739 - val_binary_crossentropy: 0.0647 - val_jaccard_coef_int: 0.7930 - val_binary_accuracy: 0.9859
Epoch 10/50
400/400 [==============================] - 40s 100ms/step - loss: 0.4647 - binary_crossentropy: 0.0661 - jaccard_coef_int: 0.7825 - binary_accura

Epoch 13/50
400/400 [==============================] - 40s 99ms/step - loss: 0.3500 - binary_crossentropy: 0.0372 - jaccard_coef_int: 0.7649 - binary_accuracy: 0.9953 - val_loss: 0.4793 - val_binary_crossentropy: 0.0353 - val_jaccard_coef_int: 0.6633 - val_binary_accuracy: 0.9961
Epoch 14/50
400/400 [==============================] - 40s 99ms/step - loss: 0.3290 - binary_crossentropy: 0.0355 - jaccard_coef_int: 0.7770 - binary_accuracy: 0.9956 - val_loss: 0.5869 - val_binary_crossentropy: 0.0434 - val_jaccard_coef_int: 0.5989 - val_binary_accuracy: 0.9954
Epoch 15/50
400/400 [==============================] - 39s 98ms/step - loss: 0.3170 - binary_crossentropy: 0.0347 - jaccard_coef_int: 0.7834 - binary_accuracy: 0.9958 - val_loss: 0.3509 - val_binary_crossentropy: 0.0243 - val_jaccard_coef_int: 0.7431 - val_binary_accuracy: 0.9970
Epoch 16/50
400/400 [==============================] - 39s 99ms/step - loss: 0.2901 - binary_crossentropy: 0.0320 - jaccard_coef_int: 0.7997 - binary_accurac

...

Epoch 26/50
400/400 [==============================] - 40s 101ms/step - loss: 0.1962 - binary_crossentropy: 0.0293 - jaccard_coef_int: 0.8956 - binary_accuracy: 0.9950 - val_loss: 0.2207 - val_binary_crossentropy: 0.0346 - val_jaccard_coef_int: 0.8828 - val_binary_accuracy: 0.9942
Epoch 27/50
400/400 [==============================] - 41s 102ms/step - loss: 0.1915 - binary_crossentropy: 0.0291 - jaccard_coef_int: 0.8978 - binary_accuracy: 0.9951 - val_loss: 0.3139 - val_binary_crossentropy: 0.0638 - val_jaccard_coef_int: 0.8378 - val_binary_accuracy: 0.9915
Epoch 28/50
400/400 [==============================] - 40s 101ms/step - loss: 0.1837 - binary_crossentropy: 0.0278 - jaccard_coef_int: 0.9018 - binary_accuracy: 0.9953 - val_loss: 0.3329 - val_binary_crossentropy: 0.0732 - val_jaccard_coef_int: 0.8289 - val_binary_accuracy: 0.9911
Epoch 29/50
400/400 [==============================] - 41s 102ms/step - loss: 0.1811 - binary_crossentropy: 0.0282 - jaccard_coef_int: 0.9033 - binary_acc

Epoch 3/50
400/400 [==============================] - 41s 102ms/step - loss: 0.6329 - binary_crossentropy: 0.0246 - jaccard_coef_int: 0.5790 - binary_accuracy: 0.9981 - val_loss: 0.4105 - val_binary_crossentropy: 0.0173 - val_jaccard_coef_int: 0.7106 - val_binary_accuracy: 0.9984
Epoch 4/50
400/400 [==============================] - 41s 102ms/step - loss: 0.5733 - binary_crossentropy: 0.0238 - jaccard_coef_int: 0.6015 - binary_accuracy: 0.9982 - val_loss: 0.3724 - val_binary_crossentropy: 0.0171 - val_jaccard_coef_int: 0.7288 - val_binary_accuracy: 0.9984
Epoch 5/50
400/400 [==============================] - 41s 101ms/step - loss: 0.5551 - binary_crossentropy: 0.0244 - jaccard_coef_int: 0.6081 - binary_accuracy: 0.9982 - val_loss: 0.5951 - val_binary_crossentropy: 0.0274 - val_jaccard_coef_int: 0.5833 - val_binary_accuracy: 0.9971
Epoch 6/50
400/400 [==============================] - 41s 101ms/step - loss: 0.5444 - binary_crossentropy: 0.0241 - jaccard_coef_int: 0.6107 - binary_accurac

Epoch 9/50
400/400 [==============================] - 51s 128ms/step - loss: 0.5632 - binary_crossentropy: 0.0995 - jaccard_coef_int: 0.7919 - binary_accuracy: 0.9804 - val_loss: 0.4826 - val_binary_crossentropy: 0.0735 - val_jaccard_coef_int: 0.8099 - val_binary_accuracy: 0.9841
Epoch 10/50
400/400 [==============================] - 51s 126ms/step - loss: 0.5139 - binary_crossentropy: 0.0945 - jaccard_coef_int: 0.8084 - binary_accuracy: 0.9820 - val_loss: 0.5353 - val_binary_crossentropy: 0.0977 - val_jaccard_coef_int: 0.7959 - val_binary_accuracy: 0.9817
Epoch 11/50
400/400 [==============================] - 51s 127ms/step - loss: 0.4858 - binary_crossentropy: 0.0936 - jaccard_coef_int: 0.8175 - binary_accuracy: 0.9830 - val_loss: 0.4583 - val_binary_crossentropy: 0.0759 - val_jaccard_coef_int: 0.8154 - val_binary_accuracy: 0.9846
Epoch 12/50
400/400 [==============================] - 51s 126ms/step - loss: 0.4546 - binary_crossentropy: 0.0903 - jaccard_coef_int: 0.8275 - binary_accu

Epoch 15/50
400/400 [==============================] - 50s 125ms/step - loss: 0.2737 - binary_crossentropy: 0.0335 - jaccard_coef_int: 0.8572 - binary_accuracy: 0.9931 - val_loss: 0.2730 - val_binary_crossentropy: 0.0246 - val_jaccard_coef_int: 0.8307 - val_binary_accuracy: 0.9949
Epoch 16/50
400/400 [==============================] - 51s 126ms/step - loss: 0.2531 - binary_crossentropy: 0.0312 - jaccard_coef_int: 0.8674 - binary_accuracy: 0.9936 - val_loss: 0.2239 - val_binary_crossentropy: 0.0188 - val_jaccard_coef_int: 0.8638 - val_binary_accuracy: 0.9957
Epoch 17/50
400/400 [==============================] - 50s 125ms/step - loss: 0.2437 - binary_crossentropy: 0.0306 - jaccard_coef_int: 0.8716 - binary_accuracy: 0.9938 - val_loss: 0.2168 - val_binary_crossentropy: 0.0185 - val_jaccard_coef_int: 0.8662 - val_binary_accuracy: 0.9960
Epoch 18/50
400/400 [==============================] - 50s 126ms/step - loss: 0.2311 - binary_crossentropy: 0.0296 - jaccard_coef_int: 0.8784 - binary_acc

400/400 [==============================] - 41s 102ms/step - loss: 0.3679 - binary_crossentropy: 0.0850 - jaccard_coef_int: 0.8816 - binary_accuracy: 0.9854 - val_loss: 0.4001 - val_binary_crossentropy: 0.0895 - val_jaccard_coef_int: 0.8783 - val_binary_accuracy: 0.9835
Epoch 21/50
400/400 [==============================] - 41s 103ms/step - loss: 0.3533 - binary_crossentropy: 0.0828 - jaccard_coef_int: 0.8864 - binary_accuracy: 0.9859 - val_loss: 0.8423 - val_binary_crossentropy: 0.3310 - val_jaccard_coef_int: 0.7537 - val_binary_accuracy: 0.9653
Epoch 22/50
400/400 [==============================] - 41s 102ms/step - loss: 0.3365 - binary_crossentropy: 0.0793 - jaccard_coef_int: 0.8911 - binary_accuracy: 0.9866 - val_loss: 0.2789 - val_binary_crossentropy: 0.0553 - val_jaccard_coef_int: 0.9134 - val_binary_accuracy: 0.9888
Epoch 23/50
400/400 [==============================] - 41s 102ms/step - loss: 0.3203 - binary_crossentropy: 0.0758 - jaccard_coef_int: 0.8954 - binary_accuracy: 0.987